In [46]:
import pandas as pd
from tqdm import tqdm

In [47]:
data_buku = pd.read_csv('data/Official.csv')

In [48]:
data_buku.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Category ID,Category
0,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,22,Romance
1,278418,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,22,Romance
2,638,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,22,Romance
3,3363,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,22,Romance
4,7158,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,22,Romance


In [49]:
data_buku.drop(["Category ID", "Year-Of-Publication", "Book-Rating", "User-ID", "ISBN"], axis=1, inplace=True)

In [50]:
data_buku.head()

,Book-Title,Book-Author,Publisher,Category
0,The Notebook,Nicholas Sparks,Warner Books,Romance
1,The Notebook,Nicholas Sparks,Warner Books,Romance
2,The Notebook,Nicholas Sparks,Warner Books,Romance
3,The Notebook,Nicholas Sparks,Warner Books,Romance
4,The Notebook,Nicholas Sparks,Warner Books,Romance


In [51]:
data_buku.drop_duplicates(subset="Book-Title",inplace=True, keep="first")

In [52]:
data_buku.head()

,Book-Title,Book-Author,Publisher,Category
0,The Notebook,Nicholas Sparks,Warner Books,Romance
116,A Painted House,JOHN GRISHAM,Doubleday,"Mystery, Thriller & Suspense"
300,Lightning,Dean R. Koontz,Berkley Publishing Group,Business & Money
434,Waiting to Exhale,Terry McMillan,Pocket,Literature & Fiction
522,Apricots on the Nile: A Memoir with Recipes,Colette Rossant,Bloomsbury Publishing Plc,"Cookbooks, Food & Wine"


In [53]:
data_buku.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7728 entries, 0 to 96492
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Book-Title   7728 non-null   object
 1   Book-Author  7728 non-null   object
 2   Publisher    7728 non-null   object
 3   Category     7728 non-null   object
dtypes: object(4)
memory usage: 301.9+ KB


In [54]:
data_buku['book_title_list'] = data_buku['Book-Title'].str.lower()
data_buku['book_author_list'] = data_buku["Book-Author"].str.lower()
data_buku['publiser_list'] = data_buku["Publisher"].str.lower()
data_buku['category_list'] = data_buku["Category"].str.lower()

In [55]:
data_buku['category_list']

0                             romance
116      mystery, thriller & suspense
300                  business & money
434              literature & fiction
522            cookbooks, food & wine
                     ...             
96486              arts & photography
96487          cookbooks, food & wine
96489          cookbooks, food & wine
96491       science fiction & fantasy
96492                          travel
Name: category_list, Length: 7728, dtype: object

In [56]:
data_buku['category_list'] = data_buku['Category'].apply(lambda x:x.replace("'", "").replace("[",
"").replace("]", ""))
data_buku['category_list'] = data_buku.category_list.apply(lambda x: x.split(', '))

In [57]:
tit = []
for title in data_buku['book_title_list']:
    tit.append(title)

In [58]:
y = 0
for i in data_buku['category_list']:
    if isinstance(i, str):
        data_buku['category_list'][y] = i.split("delimiter")
    y = y + 1

In [59]:
data_buku['category_list'] = [' '.join(map(str, l)) for l in data_buku['category_list']]

In [60]:
data_buku['category_list'].str.lower()

0                            romance
116      mystery thriller & suspense
300                 business & money
434             literature & fiction
522            cookbooks food & wine
                    ...             
96486             arts & photography
96487          cookbooks food & wine
96489          cookbooks food & wine
96491      science fiction & fantasy
96492                         travel
Name: category_list, Length: 7728, dtype: object

In [61]:
data_buku['book_author_list'] = [' '.join(map(str, l)) for l in data_buku['book_author_list']]

In [62]:
# data_buku['category_list'] = data_buku['category_list'].apply(lambda x: [x])

In [63]:
data_buku['category_list']

0                            Romance
116      Mystery Thriller & Suspense
300                 Business & Money
434             Literature & Fiction
522            Cookbooks Food & Wine
                    ...             
96486             Arts & Photography
96487          Cookbooks Food & Wine
96489          Cookbooks Food & Wine
96491      Science Fiction & Fantasy
96492                         Travel
Name: category_list, Length: 7728, dtype: object

In [64]:
data_buku['book_author_list'] = [' '.join(map(str, l)) for l in data_buku['book_author_list']]

In [65]:
data_buku['soup'] = data_buku['book_title_list']

In [66]:
import string
import re
import nltk

In [67]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/iqbalf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [68]:
# menghapus tab, new line, backslice, non ASCII dan incomplete URL
data_buku['soup'] = data_buku['soup'].apply(lambda x: x.replace('\\t'," ").replace('\\n',"").replace('\\u'," ").replace('\\',"").replace("http://", " ").replace("https://"," "))
data_buku['soup'] = data_buku['soup'].apply(lambda x: x.encode('ascii', 'replace').decode('ascii'))
data_buku['soup'] = data_buku['soup'].apply(lambda x: ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", x).split()))

 # menghapus angka
data_buku['soup'] = data_buku['soup'].apply(lambda x: re.sub(r"\d+", "", x))
 # replace simbol with space
data_buku['soup'] = data_buku['soup'].apply(lambda x: x.translate({ord(c): " " for c in "!@$%^&*()[]{};:,./<>?\|`~-=_+—"}))
 # menghapus tanda baca
data_buku['soup'] = data_buku['soup'].apply(lambda x: x.translate(str.maketrans("","",string.punctuation)))
 # menghapus whitespace leading & trailing
data_buku['soup'] = data_buku['soup'].apply(lambda x: x.strip())

 # mengganti multiple whitespace menjadi single whitespace
data_buku['soup'] = data_buku['soup'].apply(lambda x: re.sub('\s+',' ', x))
 # menghapus single char
data_buku['soup'] = data_buku['soup'].apply(lambda x: re.sub(r"\b[a-zA-Z]\b", "", x))
# NLTK word tokenize
data_buku['soup'] = data_buku['soup'].apply(lambda x: word_tokenize(x))

In [69]:
# mendownload daftar kata yang ada
nltk.download('stopwords')
from nltk.corpus import stopwords

# memeriksa daftar kata di stopwords
indo = stopwords.words('indonesian')
eng = stopwords.words('english')

# list stopwords bahasa indonesia
list_stopwords = indo

# menambah stopwords bahasa inggris
list_stopwords.extend(eng)

# hapus stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data_buku['soup'] = data_buku['soup'].apply(stopwords_removal)

[nltk_data] Downloading package stopwords to /home/iqbalf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}


In [71]:
data_buku['soup'] = data_buku['soup'].replace(['bonesetters daughter'], 'bonesetter')

In [72]:
result = data_buku[data_buku["soup"] == "bonesetters daughter"]
result

,Book-Title,Book-Author,Publisher,Category,book_title_list,book_author_list,publiser_list,category_list,soup


In [73]:
# for document in data_buku['soup']:
#     for term in document:
#         if term not in term_dict:
#             term_dict[term] = ' '

# print(len(term_dict))
# print("------------------------")

# for term in term_dict:
#     term_dict[term] = stemmed_wrapper(term)
#     # print(term,":" ,term_dict[term])

# print(term_dict)
# print("------------------------")

# # apply stemmed term to dataframe
# def get_stemmed_term(document):
#     return [term_dict[term] for term in document]

In [74]:
import swifter
import pickle5 as pickle

# data_buku['soup'] = data_buku['soup'].swifter.apply(get_stemmed_term)
# pickle.dump(data_buku, open('model/cek1.pkl', 'wb'))
data_buku = pickle.load(open('model/cek1.pkl', 'rb'))
# mengubah list pada kolom soup menjadi string
data_buku['soup'] = data_buku['soup'].agg(lambda x: ' '.join(map(str, x)))
# menggabungkan soup sebelumnya dengan author dan genre


In [75]:
data_buku.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7728 entries, 0 to 96492
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Book-Title        7728 non-null   object
 1   Book-Author       7728 non-null   object
 2   Publisher         7728 non-null   object
 3   Category          7728 non-null   object
 4   book_title_list   7728 non-null   object
 5   book_author_list  7728 non-null   object
 6   publiser_list     7728 non-null   object
 7   category_list     7728 non-null   object
 8   soup              7728 non-null   object
dtypes: object(9)
memory usage: 603.8+ KB


In [76]:
data_buku['category_list']

0                            Romance
116      Mystery Thriller & Suspense
300                 Business & Money
434             Literature & Fiction
522            Cookbooks Food & Wine
                    ...             
96486             Arts & Photography
96487          Cookbooks Food & Wine
96489          Cookbooks Food & Wine
96491      Science Fiction & Fantasy
96492                         Travel
Name: category_list, Length: 7728, dtype: object

In [77]:
# data_buku['book_author_list'] = ' '.join(str(x) for x in data_buku['book_author_list'])

In [78]:
# data_buku['category_list'] = ' '.join(str(x) for x in data_buku['category_list'])

In [79]:
data_buku['category_list'].str.lower()

0                            romance
116      mystery thriller & suspense
300                 business & money
434             literature & fiction
522            cookbooks food & wine
                    ...             
96486             arts & photography
96487          cookbooks food & wine
96489          cookbooks food & wine
96491      science fiction & fantasy
96492                         travel
Name: category_list, Length: 7728, dtype: object

In [80]:
data_buku['category_list'] = [' '.join(map(str, l)) for l in data_buku['category_list']]

In [81]:
data_buku['soup'] = data_buku['soup']+ " " + data_buku['book_author_list']

In [82]:
data_buku['soup'] = data_buku['soup']+ " " + data_buku['category_list']

In [83]:
# data_buku.to_csv('data/soup.csv', index=False)

In [84]:
# data_buku['soup'] = data_buku['soup'] + " " + data_buku['category_list']

In [85]:
# libraries for Recommendation System
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words=list_stopwords)

# membangun matriks TF-IDF
tfidf_matrix = tfidf.fit_transform(data_buku['soup'])

# output the shape of tfidf_matrix
tfidf_matrix.shape

/home/iqbalf/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


(7728, 8971)

In [86]:
# import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# menghitung matriks cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# reset index dataframe
data_buku = data_buku.reset_index()
titles = data_buku['Book-Title']

# membuat map dari index dan judul buku
indices = pd.Series(data_buku.index, index=data_buku['Book-Title']).drop_duplicates()

In [87]:
def rec_tfidf(title, cosine_sim=cosine_sim):
    recommendation = pd.DataFrame(columns = ['Book Idx', 'Title', 'Score'])
    count = 0

    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    book_indices = [i[0] for i in sim_scores]

    for i in book_indices:
        recommendation.at[count, 'Book Idx'] = book_indices[count]
        recommendation.at[count, 'Title'] = titles.iloc[book_indices[count]]
        recommendation.at[count, 'Score'] = sim_scores[count][1]
        count += 1
    return recommendation

In [88]:
rec_tfidf('A Painted House')

,Book Idx,Title,Score
0,2019,The Painted Word,0.610007
1,979,House,0.568853
2,1691,The House,0.568853
3,4114,The Little House,0.40364
4,907,Open House,0.360715


In [89]:
def get_recommendations(judul, jumlah):
# Creates Series
    judul = pd.Series(judul)
    judul = judul.apply(lambda x: re.sub(r"\d+", "", x))
    judul = judul.apply(lambda x: x.translate({ord(c): " " for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+—"}))
    judul = judul.apply(lambda x: x.strip())
    judul = judul.apply(lambda x: re.sub('\s+',' ', x))
    judul = judul.apply(lambda x: re.sub(r"\b[a-zA-Z]\b", "", x))

    judul = judul.apply(lambda x: word_tokenize(x))

    # Ambil kata kunci
    judul = judul.to_string()
    judul = judul[6:-1]
    judul = re.sub(r'[^\w\s]', '', judul)
    # Membaca data
    books = data_buku
    gnr = "['keywords', {judul}]"
    data = [{'Unnamed: 0': 5143, 'Book-Title': judul, 'Book-Author': judul, 'Category': gnr,  'book_title_list': judul, 'book_author_list': judul, 'category_list': judul, 'soup': judul}]
    # books.drop_duplicates(subset ="title", inplace = True)
    # Menggabungkan database buku dengan kata kunci yang dimasukkan
    new = pd.DataFrame(data)
    frames = [books, new]
    result = pd.concat(frames)

    books = result.copy()

    tfidf = TfidfVectorizer(stop_words=list_stopwords)
    tfidf_matrix = tfidf.fit_transform(books['soup'])
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    books = books.reset_index()
    titles = books['Book-Title']
    titles = titles.str.lower()
    authors = books['Book-Author']
    # Implementasi kode pada Gambar 4.7 (tanpa set genre menjadi lowercase)
    books['Category'] = [', '.join(map(str, l)) for l in books['Category']]
    genres = books['Category']
    indices = pd.Series(books.index, index=titles)
    # Memebentuk tampilan dataframe berdasarkan judul, penulis, genre dan cover
    jumlah = int(jumlah) + 1
    rec = pd.DataFrame(columns = ['Judul', 'Penulis', 'Genre'])
    count = 0
    idx = indices[judul]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:jumlah]
    book_indices = [i[0] for i in sim_scores]
    for i in book_indices:
        rec.at[count+1, 'Judul'] = titles.iloc[book_indices[count]].title()
        rec.at[count+1, 'Penulis'] = authors.iloc[book_indices[count]].title()
        rec.at[count+1, 'Genre'] = genres.iloc[book_indices[count]].title()
        count += 1
    return rec

In [93]:
get_recommendations('mystery', 5)

/home/iqbalf/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


,Judul,Penulis,Genre
1,Magic And Mystery In Tibet,Alexandra David-Neel,"T, R, A, V, E, L"
2,The Mystery Of Agatha Christie,Gwen Robyns,"M, Y, S, T, E, R, Y, ,, , T, H, R, I, L, L, E..."
3,Bunnicula: A Rabbit-Tale Of Mystery,Deborah And James Howe,"C, H, I, L, D, R, E, N, ', S, , B, O, O, K, S"
4,She Who Is: The Mystery Of God In Feminist The...,Elizabeth A. Johnson,"P, O, L, I, T, I, C, S, , &, , S, O, C, I, A..."
5,The St. Patrick'S Day Shamrock Mystery,Marion Markham,"C, H, I, L, D, R, E, N, ', S, , B, O, O, K, S"
